In [99]:
import sys
sys.path.append('..')
from modified_sage_funcs import *
from imp_funcs import *

r, n = 2, 4
M = matroids.Uniform(r, n)
d = M.full_rank()
S = (frozenset({0}), frozenset({0}), frozenset({1}), frozenset({0}), frozenset({1}), frozenset({2}))
k = len(S)

bases = sorted([frozenset(sorted(X)) for X in bases_iterator(M)])
bases

[frozenset({0, 1}),
 frozenset({0, 2}),
 frozenset({0, 3}),
 frozenset({1, 2}),
 frozenset({1, 3}),
 frozenset({2, 3})]

In [100]:
indexed_compositions = {i : x for i, x in enumerate(weak_compositions(d, k))}
indexed_compositions

{0: [1, 1, 0, 0, 0, 0],
 1: [1, 0, 1, 0, 0, 0],
 2: [1, 0, 0, 1, 0, 0],
 3: [1, 0, 0, 0, 1, 0],
 4: [1, 0, 0, 0, 0, 1],
 5: [0, 1, 1, 0, 0, 0],
 6: [0, 1, 0, 1, 0, 0],
 7: [0, 1, 0, 0, 1, 0],
 8: [0, 1, 0, 0, 0, 1],
 9: [0, 0, 1, 1, 0, 0],
 10: [0, 0, 1, 0, 1, 0],
 11: [0, 0, 1, 0, 0, 1],
 12: [0, 0, 0, 1, 1, 0],
 13: [0, 0, 0, 1, 0, 1],
 14: [0, 0, 0, 0, 1, 1],
 15: [2, 0, 0, 0, 0, 0],
 16: [0, 2, 0, 0, 0, 0],
 17: [0, 0, 2, 0, 0, 0],
 18: [0, 0, 0, 2, 0, 0],
 19: [0, 0, 0, 0, 2, 0],
 20: [0, 0, 0, 0, 0, 2]}

In [101]:
from itertools import permutations

def partitions(basis, weak_comp):
    parts = set()
    for perm in permutations(basis):
        partition = []
        start = 0
        for size in weak_comp:
            partition.append(frozenset(perm[start:start + size]))
            start += size
        parts.add(tuple(partition))
    return tuple(parts)

def base_partitions(bases, indexed_compositions):
    bases_partitions = {}
    for basis in bases:
        base_partitions = {}
        for i in indexed_compositions:
            base_partitions[i] = partitions(tuple(basis), indexed_compositions[i])
        bases_partitions[basis] = base_partitions
    return bases_partitions

base_partitions = base_partitions(bases, indexed_compositions)


In [102]:
def composition_weight(iwc, set_system):
    weights = {}
    
    for i in iwc:
        weight = 0
        for basis in bases:
            for basis_partition in base_partitions[basis][i]:
                flag = True
                for x, y, z in zip(iwc[i], basis_partition, set_system):
                    if len(y.intersection(z)) == x:
                        continue
                    else:
                        flag = False
                        break
                if flag:
                    weight += 1
                    break
        weights[i] = weight
    return weights

weights = composition_weight(indexed_compositions, S)

weights

{0: 0,
 1: 1,
 2: 0,
 3: 1,
 4: 1,
 5: 1,
 6: 0,
 7: 1,
 8: 1,
 9: 1,
 10: 0,
 11: 1,
 12: 1,
 13: 1,
 14: 1,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0,
 20: 0}

In [103]:
P = PolynomialRing(QQ, 'x', k)

def monomial(wc):
    return prod([P.gen(i)**wc[i] for i in range(len(wc))])/prod([factorial(x) for x in wc])

def monomial2(wc):
    return prod([P.gen(i)**wc[i] for i in range(len(wc))])

def normalized_weighted_sum(weights):
    return sum([weights[i]*monomial(indexed_compositions[i]) for i in weights])

def weighted_sum(weights):
    return sum([weights[i]*monomial2(indexed_compositions[i]) for i in weights])

print(indexed_compositions)
poly = weighted_sum(composition_weight(indexed_compositions, S))
poly.factor()

{0: [1, 1, 0, 0, 0, 0], 1: [1, 0, 1, 0, 0, 0], 2: [1, 0, 0, 1, 0, 0], 3: [1, 0, 0, 0, 1, 0], 4: [1, 0, 0, 0, 0, 1], 5: [0, 1, 1, 0, 0, 0], 6: [0, 1, 0, 1, 0, 0], 7: [0, 1, 0, 0, 1, 0], 8: [0, 1, 0, 0, 0, 1], 9: [0, 0, 1, 1, 0, 0], 10: [0, 0, 1, 0, 1, 0], 11: [0, 0, 1, 0, 0, 1], 12: [0, 0, 0, 1, 1, 0], 13: [0, 0, 0, 1, 0, 1], 14: [0, 0, 0, 0, 1, 1], 15: [2, 0, 0, 0, 0, 0], 16: [0, 2, 0, 0, 0, 0], 17: [0, 0, 2, 0, 0, 0], 18: [0, 0, 0, 2, 0, 0], 19: [0, 0, 0, 0, 2, 0], 20: [0, 0, 0, 0, 0, 2]}


x0*x2 + x1*x2 + x2*x3 + x0*x4 + x1*x4 + x3*x4 + x0*x5 + x1*x5 + x2*x5 + x3*x5 + x4*x5

In [104]:
poly.is_lorentzian()

True